<a href="https://colab.research.google.com/github/vlykamol/GrowOwn/blob/main/CBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
grid = []
m, n = 0, 0

In [ ]:
# taking grid input
def gridInput():
    try:
        # Get the dimensions of the grid (m x n)
        m = int(input("Enter the number of rows (m): "))
        n = int(input("Enter the number of columns (n): "))

        # Initialize an empty grid
        grid = []

        # Loop to get the grid elements
        for i in range(m):
            row = input(f"Enter row {i + 1} (use 0 for empty, 1 for obstacle, separated by spaces): ").split()
            if len(row) != n:
                raise ValueError("Invalid input! Please enter the correct number of elements for each row.")
            row = [int(cell) for cell in row]
            grid.append(row)

        # Return the grid
        return grid, m, n

    except ValueError as e:
        print(f"Error: {e}")
        return None

grid, m, n = gridInput()

In [ ]:
#printing grid
if grid:
    print("Grid:")
    for row in grid:
        print(row)

# all the definations

In [ ]:
class Location:
      def __init__(self, x, y):
            self.x = x
            self.y = y
      def __eq__(self, other):
            return self.x == other.x and self.y == other.y
      def __str__(self):
            return f"at {self.x}, {self.y}"
      def __hash__(self):
            return hash(str(self.x) + str(self.y))

class Constraint():
      def __init__(self, time, location):
            self.time = time
            self.location = location
      def __eq__(self, other):
            return self.time == other.time and self.location == other.location
      def __hash__(self):
            return hash(str(self.time) + str(self.location.x) + str(self.location.y))

class State():
      def __init__(self, time, location):
            self.time = time
            self.location = location
      def __eq__(self, other):
            return self.time == other.time and self.location == other.location
      def __hash__(self):
            return hash(str(self.time)+str(self.location))
      def __str__(self):
        return f"state(time={self.time}, location={self.location})"


In [ ]:
agents = {}
a = 0
# taking agents start and end locations
def agentInput():
  try:
    # get number of agents (a)
    a = int(input("Enter the number of agents"))

    # Loop to get the agents
    for i in range(a):
      startX, startY  = input(f"Enter agent {i + 1} start location (row, col space seprated): ").split()
      startX = int(startX)
      startY = int(startY)
      if startX >= m or startX < 0 or startY >= n or startY < 0 :
        raise ValueError(f"Invalid input! Please enter the correct start location for agent {i + 1}.")
      endX, endY  = input(f"Enter agent {i + 1} end location (row, col space seprated): ").split()
      endX = int(endX)
      endY = int(endY)
      if endX >= m or endX < 0 or endY >= n or endY < 0 :
        raise ValueError(f"Invalid input! Please enter the correct end location for agent {i + 1}.")
      start = Location(startX,startY)
      end = Location(endX,endY)
      agents[f"R{i + 1}"] = {"start" : start, "end" : end}
    return a

  except ValueError as e:
    print(f"Error: {e}")
    return None

a = agentInput()

In [ ]:
packages = {}
# taking packages start and end locations
def packageInput():
  try:
    # Loop to get the packages
    for i in range(a):
      startX, startY  = input(f"Enter package {i + 1} start location (row, col space seprated): ").split()
      startX = int(startX)
      startY = int(startY)
      if startX >= m or startX < 0 or startY >= n or startY < 0 :
        raise ValueError(f"Invalid input! Please enter the correct start location for package {i + 1}.")
      endX, endY  = input(f"Enter package {i + 1} end location (row, col space seprated): ").split()
      endX = int(endX)
      endY = int(endY)
      if endX >= m or endX < 0 or endY >= n or endY < 0 :
        raise ValueError(f"Invalid input! Please enter the correct end location for package {i + 1}.")
      start = Location(startX, startY)
      end = Location(endX, endY)
      packages[f"P{i + 1}"] = {"start" : start, "end" : end}

    return packages

  except ValueError as e:
    print(f"Error: {e}")
    return None

packageInput()

In [ ]:
_grid = [row[:] for row in grid]
for i in agents:
  _a = agents[i]
  start = _a['start']
  end = _a['end']
  _grid[start.x][start.y] = i
  _grid[end.x][end.y] = f"E{i}"
  print(i, f"starts {start}", f"ends {end}")
for i in packages:
  _a = packages[i]
  start = _a['start']
  end = _a['end']
  _grid[start.x][start.y] = i
  _grid[end.x][end.y] = f"D{i}"
  print(i, f"starts {start}", f"ends {end}")

In [ ]:
for row in _grid:
  print(row)

In [ ]:
def valid_state(location):
  return location.x >= 0 and location.x < m and location.y >= 0 and location.y < n and grid[location.x][location.y] != 1


def get_successors(current_node):
  print("current node", current_node, agents)
  neighbors = []
  time = current_node.time
  location = current_node.location
  # Wait action
  n = Constraint(time + 1, location)
  if valid_state(location):
      neighbors.append(n)
  # Up action
  n = Constraint(time + 1, Location(location.x, location.y+1))
  if valid_state(n):
      neighbors.append(n)
  # Down action
  n = Constraint(time + 1, Location(location.x, location.y-1))
  if valid_state(n):
      neighbors.append(n)
  # Left action
  n = Constraint(time + 1, Location(location.x-1, location.y))
  if valid_state(n):
      neighbors.append(n)
  # Right action
  n = Constraint(time + 1, Location(location.x+1, location.y))
  if valid_state(n):
      neighbors.append(n)
  return neighbors


In [ ]:
#calculating Manhattan distance
def manhattan(current_node, goal_node, grid):
    print("current node", current_node.location)

    x1, y1 = current_node.location.x, current_node.location.y
    x2, y2 = goal_node.location.x, goal_node.location.y

    # Manhattan distance, considering blocked cells
    dist = 0
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    if x1 < x2:
        for x in range(x1, x2):
            if grid[x][y1] == 1:
                dist += 1
    else:
        for x in range(x2, x1):
            if grid[x][y1] == 1:
                dist += 1

    if y1 < y2:
        for y in range(y1, y2):
            if grid[x2][y] == 1:
                dist += 1
    else:
        for y in range(y2, y1):
            if grid[x2][y] == 1:
                dist += 1

    return dist + dx + dy

#calculating heuristic
def heuristic(agent, packages):
    _agent = agents[agent]
    start = _agent['start']
    end = _agent['end']

    heuristics = {}
    for _p in packages:
        p = packages[_p]
        pstart = p['start']
        pend = p['end']
        path = [start, pstart, pend, end]
        dist = manhattan(path[0], path[1], grid) + manhattan(path[1], path[2], grid) + manhattan(path[2], path[3], grid)
        heuristics.update({_p : dist})

    package = min(heuristics, key=heuristics.get)
    min_heuristic = heuristics[package]
    return package, min_heuristic

In [ ]:
def isGoal(current, agent):
  print("is goal",agent)

In [ ]:
# A* search
def search(agent):
  print("befor initial state", agent)
  initial_state = agents[agent]["start"]
  print("after initial state", initial_state.time, initial_state.location)
  step_cost = 1

  closed_set = set()
  open_set = {initial_state}

  came_from = {}

  g_score = {}
  g_score[initial_state] = 0

  f_score = {}
  package, min_heuristc = heuristic(agent, packages)
  f_score[initial_state] = {min_heuristc : package}

  while open_set:
    temp_dict = {open_item:f_score.setdefault(open_item, float("inf")) for open_item in open_set}
    current = min(temp_dict, key=temp_dict.get)

    print("temp_dict", temp_dict)
    print("current min", current)

    if isGoal(current, agent):
        return constructPath(came_from, current)

    open_set -= {current}
    closed_set |= {current}

    neighbor_list = get_successors(current)

    for neighbor in neighbor_list:
      if neighbor in closed_set:
          continue

    # calculate g_score and f_score for all neighbour
  return False

In [ ]:
def calculate_solution():
        solution = {}
        for agent in agents.keys():
            _a = agents[agent]
            start = State(0, _a["start"])
            end = State(0, _a["end"])

            agents[agent] = {"start" : start, "end" : end}
            local_solution = search(agent)
            print("local solution")
            print(local_solution)
            if not local_solution:
                return False
            solution.update({agent:local_solution})
        return solution

def total_cost(self, solution):
    return sum([len(path) for path in solution.values()])

In [ ]:
class Conflict():
  def __init__(self, time, agent1, agent2):
    self.time = time
    self.agent1 = agent1
    self.agent2 = agent2

def get_conflict():
  pass

In [ ]:
solutions = {}
Constraints = {}
cost = 0

class AgentNode():
   def __init__(self):
    self.solution = {}
    self.constraint_dict = {}
    self.cost = 0

class CBS:
  def __init__(self):
    self.open_set = set()
    self.closed_set = set()
#low level search
  def search(self):
    rootNode = AgentNode()
    rootNode.constraint_dict = {}
    for a in agents.keys():
       rootNode.constraint_dict[a] = {}
    solutions = calculate_solution() # try to calculate solution for all possible options
    print(solutions)
    if not solutions:
        return {}
    cost = total_cost(solutions)

#high level search
    self.open_set |= {rootNode}

    print(self.open_set)

    while self.open_set:
       # find conflicts and resolve

In [ ]:
cbs = CBS()
solution = cbs.search()